In [2]:
pip install torch

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   - -------------------------------------- 6.0/204.1 MB 32.3 MB/s eta 0:00:07
   -- ------------------------------------- 13.9/204.1 MB 34.2 MB/s eta 0:00:06
   ---- ----------------------------------- 21.2/204.1 MB 34.9 MB/s eta 0:00:06
   ----- ---------------------------------- 29.1/204.1 MB 36.0 MB/s eta 0:00:05
   ------- -------------------------------- 36.7/204.1 MB 36.4 MB/s eta 0:00:05
   -------- ------------------------------- 45.4/204.1 MB 36.9 MB/s eta 0:00:05
   ---------- ----------------------------- 53.0/204.1 MB 36.9 MB/s eta 0:00:05
   ----------- ---------------------------- 60.6/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install kagglehub

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------------- ------------------ 6.3/11.5 MB 36.1 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 32.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------------------------- ------------- 8.4/12.6 MB 39.4 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 35.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
pip install PIL

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for PIL


In [102]:
pip install pillow

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 14.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [107]:
import kagglehub
import torch
import pandas as pd
import os
from pathlib import Path
from PIL import Image
import numpy as np

In [94]:
path = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
car_pic = os.path.join(path, r'TTV\pred')

In [ ]:
for image_path in os.listdir(car_pic):
    input_path = os.path.join(car_pic, image_path)
    img = Image.open(input_path)